In [1]:
import gensim
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from wordcloud import WordCloud
from collections import Counter
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import spacy
import string
import gensim.downloader as api
print(list(gensim.downloader.info()['models'].keys()))

wv = api.load('glove-twitter-50')

# Download the stopwords dataset
nltk.download('stopwords')

# Download required resources
nltk.download('wordnet')
nltk.download('omw-1.4')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Test the lemmatizer with a simple example
print(lemmatizer.lemmatize('running'))  # Should output 'run'

ImportError: cannot import name 'triu' from 'scipy.linalg' (c:\Users\wware\AppData\Local\anaconda3\envs\fake-news-predictor\lib\site-packages\scipy\linalg\__init__.py)

In [2]:
def handle_bad_line(bad_lines):
    # Custom logic to handle or log bad lines
    print(f"Bad line: {bad_lines}")

df = pd.read_csv('/content/drive/MyDrive/data/WELFake_Dataset.csv', on_bad_lines=handle_bad_line, engine='python')

Bad line: ['Scandinavian countries can do ok with socialism because of their high functioning populace. They’re smarter and have a better culture in terms of work ethic', ' norms about work', ' etc', ' on average. They also live in tiny homes and drive tiny cars to achieve some of this “prosperity”. They also don’t have much military spending', ' which is why they were conquered in a few weeks the last time anyone tried. A uniform ethnic makeup has helped with cultural issues of socialism as well', ' though this is changing. ']
Bad line: ['Вы сейчас мне скажете', ' Вы сказали: в Крыму. А что в Крыму? Что Вы сделали в Югославии и потом отделили от неё часть государств', ' в том числе и Косово', ' а потом от Сербии. В Косово парламент проголосовал после военных действий интервенции', ' после многочисленных жертв. И всё', ' решение принято', ' и все с этим согласились.']
Bad line: ['В Крыму не было никакой войны. Не было никаких бомбовых ударов', ' военных действий', ' жертв нет', ' вообщ

In [3]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [4]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [5]:
df.shape

(72154, 3)

In [6]:
# Combine 'title' and 'text' into a single column called 'combined_text'
df['combined_text'] = df['title'].fillna('') + " " + df['text']
df.head()

,title,text,label,combined_text
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,LAW ENFORCEMENT ON HIGH ALERT Following Threat...
1,NaN,Did they post their votes for Hillary already?,1,Did they post their votes for Hillary already?
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,"Bobby Jindal, raised Hindu, uses story of Chri..."
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,SATAN 2: Russia unvelis an image of its terrif...


In [7]:
# Fill missing values in 'title' column with 'No Title'
df['combined_text'].fillna('No Title', inplace=True)


In [8]:
# Set of English stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    # Tokenize the text by splitting on whitespace
    word_tokens = text.split()

    # Filter out the stopwords
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]

    # Join the filtered words back into a string
    return ' '.join(filtered_text)

# Apply the function to the 'combined_text' column
df['cleaned_text'] = df['combined_text'].apply(remove_stopwords)

# Display a few examples to verify the result
print(df[['combined_text', 'cleaned_text']].head())

                                       combined_text  \
0  LAW ENFORCEMENT ON HIGH ALERT Following Threat...   
1     Did they post their votes for Hillary already?   
2  UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...   
3  Bobby Jindal, raised Hindu, uses story of Chri...   
4  SATAN 2: Russia unvelis an image of its terrif...   

                                        cleaned_text  
0  LAW ENFORCEMENT HIGH ALERT Following Threats C...  
1                        post votes Hillary already?  
2  UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS CH...  
3  Bobby Jindal, raised Hindu, uses story Christi...  
4  SATAN 2: Russia unvelis image terrifying new ‘...  


In [9]:
def clean_text(text):
    # Remove special characters and numerics using regex
    text = re.sub(r'[^A-Za-z\s]', '', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply the cleaning function to the 'cleaned_text' column
df['cleaned_text'] = df['cleaned_text'].apply(clean_text)

# Display a few examples to verify the result
print(df[['combined_text', 'cleaned_text']].head())

                                       combined_text  \
0  LAW ENFORCEMENT ON HIGH ALERT Following Threat...   
1     Did they post their votes for Hillary already?   
2  UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...   
3  Bobby Jindal, raised Hindu, uses story of Chri...   
4  SATAN 2: Russia unvelis an image of its terrif...   

                                        cleaned_text  
0  LAW ENFORCEMENT HIGH ALERT Following Threats C...  
1                         post votes Hillary already  
2  UNBELIEVABLE OBAMAS ATTORNEY GENERAL SAYS CHAR...  
3  Bobby Jindal raised Hindu uses story Christian...  
4  SATAN Russia unvelis image terrifying new SUPE...  


In [10]:
df.head()

,title,text,label,combined_text,cleaned_text
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,LAW ENFORCEMENT HIGH ALERT Following Threats C...
1,NaN,Did they post their votes for Hillary already?,1,Did they post their votes for Hillary already?,post votes Hillary already
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,UNBELIEVABLE OBAMAS ATTORNEY GENERAL SAYS CHAR...
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,"Bobby Jindal, raised Hindu, uses story of Chri...",Bobby Jindal raised Hindu uses story Christian...
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,SATAN 2: Russia unvelis an image of its terrif...,SATAN Russia unvelis image terrifying new SUPE...


In [11]:
# Perform a simple processing of text and seperated by commas
df['cleaned_text'] = df.cleaned_text.apply(gensim.utils.simple_preprocess)
df['cleaned_text']

,cleaned_text
0,"[law, enforcement, high, alert, following, thr..."
1,"[post, votes, hillary, already]"
2,"[unbelievable, obamas, attorney, general, says..."
3,"[bobby, jindal, raised, hindu, uses, story, ch..."
4,"[satan, russia, unvelis, image, terrifying, ne..."
...,...
72149,"[russians, steal, research, trump, hack, us, d..."
72150,"[watch, giuliani, demands, democrats, apologiz..."
72151,"[migrants, refuse, leave, train, refugee, camp..."
72152,"[trump, tussle, gives, unpopular, mexican, lea..."


In [12]:
# Create the Word2Vec model
model = gensim.models.Word2Vec(
    window=10, # Considers 10 words before and 10 words after each selected word in a window
    min_count=3, # Minimum words in a sentence is 3. Sentences less than that will be ignored
    workers=4, # Number of CPU threads. I used the standard in the example I have reviewed.
)

In [13]:
# Building a vocabulary showing progress for every 10,000 records
model.build_vocab(df['cleaned_text'], progress_per=1000)

In [14]:
model.corpus_count

72154

In [15]:
model.epochs

5

In [16]:
# Training the dataset to Word2Vec model
model.train(df['cleaned_text'], total_examples=model.corpus_count, epochs=model.epochs)

(110353444, 113646770)

In [17]:
model.save("word2vec_fakenews_model.model")

In [18]:
model.wv.most_similar("fake")

[('fbits', 0.6445296406745911),
 ('redflag', 0.6237335801124573),
 ('bno', 0.6233058571815491),
 ('buzzfeed', 0.5938194394111633),
 ('newswires', 0.5849853754043579),
 ('phony', 0.5742443799972534),
 ('outlet', 0.5682799220085144),
 ('false', 0.5676180720329285),
 ('bogus', 0.5554470419883728),
 ('incessant', 0.552409291267395)]

In [19]:
model.wv.most_similar("true")

[('indeed', 0.6889272928237915),
 ('truth', 0.6753630638122559),
 ('is', 0.6444991230964661),
 ('authentic', 0.6265916228294373),
 ('fact', 0.620887279510498),
 ('pure', 0.6206578612327576),
 ('all', 0.6158981323242188),
 ('essence', 0.6094827055931091),
 ('believe', 0.6058717966079712),
 ('exists', 0.599737286567688)]

In [20]:
model.wv.most_similar("news")

[('newsit', 0.7641264796257019),
 ('newsthe', 0.7406474947929382),
 ('newswatch', 0.7085038423538208),
 ('newss', 0.6958605647087097),
 ('newstrump', 0.6572794914245605),
 ('mccordsville', 0.6498075723648071),
 ('newsi', 0.6402439475059509),
 ('newsand', 0.6266341805458069),
 ('newsmeanwhile', 0.6113986968994141),
 ('newsthis', 0.6003609895706177)]

In [21]:
model.wv.similarity(w1="fake", w2="true")

0.28767416

In [22]:
model.wv.similarity(w1="true", w2="true")

1.0

In [23]:
model.wv.similarity(w1="president", w2="trump")

0.4219517

In [24]:
model.wv.similarity(w1="say", w2="said")

0.60033745

In [25]:
# Create a function for converting word/sentence to vectors

def sent_vec(sent):
    vector_size = wv.vector_size
    wv_res = np.zeros(vector_size)
    # print(wv_res)
    ctr = 1
    for w in sent:
        if w in wv:
            ctr += 1
            wv_res += wv[w]
    wv_res = wv_res/ctr
    return wv_res

In [26]:
df['vec'] = df['cleaned_text'].apply(sent_vec)

In [27]:
df.head()

,title,text,label,combined_text,cleaned_text,vec
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,"[law, enforcement, high, alert, following, thr...","[0.16382515225518893, 0.31178063951824053, 0.1..."
1,NaN,Did they post their votes for Hillary already?,1,Did they post their votes for Hillary already?,"[post, votes, hillary, already]","[0.4774920165538788, 0.3394913986325264, 0.343..."
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"[unbelievable, obamas, attorney, general, says...","[0.044578833505511284, 0.08410444772905773, 0...."
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,"Bobby Jindal, raised Hindu, uses story of Chri...","[bobby, jindal, raised, hindu, uses, story, ch...","[0.25509209773058156, 0.19708892731575753, 0.1..."
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,SATAN 2: Russia unvelis an image of its terrif...,"[satan, russia, unvelis, image, terrifying, ne...","[0.07700776124982694, 0.2894895892309886, -0.0..."


In [28]:
X = df['vec'].to_list()
y = df['label'].to_list()

In [44]:
print(np.any([x is None for x in X]))
print(np.any([y is None for y in y]))

False
True


In [41]:
X = np.array(X)
y = np.array(y)